# Buenas Prácticas (SQL)

## Introducción

Esta guía tiene como objetivo mantener la coherencia de los artefactos SQL en todo. Si bien a menudo se hace  referencia a esta guía en nuestras revisiones de código, es importante tener en cuenta que, en última instancia, esta es una guía y habrá casos de uso de piezas que no serán consistentes
con estas recomendaciones.

## Guía de Codificación

## Palabras y funciones reservadas

Utilice siempre mayúsculas para palabras clave reservadas, funciones y funciones definidas por el usuario (`UDF`) como `SELECT`, `WHERE` o `AS`.

## Nombres de variables

* No mezcle idiomas (y prefiera el inglés).
* Utilice nombres del dominio (si son tiendas, ocupe "store", si son productos, ocupe "product", si son items, ocupe "item").  * Evite crear nuevos nombres para cosas que ya existen (a esto le llamamos mantener un lenguaje ubicuo), si tiene dudas, pregunte.
* Utilice identificadores y nombres coherentes y descriptivos.
* Muchos nombres del lago de datos suelen ser versiones abreviadas, como **first_nm** vs **first_name**. Debe elegir si su tabla utilizará nombres breves o detallados y será coherente en todas las columnas de su tabla.
* Utilice nombres en minúsculas con guiones bajos, como **first_name**. No use **CamelCase**.
* Los nombres deben comenzar con una letra y no pueden terminar con un guión bajo.
* Utilice solo letras, números y guiones bajos en los nombres de las variables.

## Comentando

Evite los comentarios excesivos y haga todo lo posible para resumir tablas y filtros y explicar cada decisión/línea que pueda no estar clara.

## Sé conciso


* Apunta a la elegancia y escribe las consultas más breves y eficientes posibles y evita la duplicación.
* Las tablas temporales y el flujo de consultas deben seguir una progresión lógica (ejemplo: conjunto de datos grande filtrado en un subconjunto, filtrado o combinado con un subconjunto diferente).

## Bloques de código

Las palabras clave raíz deben estar en su propia línea. En el caso de que haya solo un argumento después de una palabra clave, la palabra clave debe
incluirse en la misma línea. En el caso de múltiples argumentos, deben ser líneas separadas desde el segundo elemento.

**bad**
```
SELECT client_id, submission_date
FROM main_summary
WHERE submission_date > '20180101' AND sample_id = '42'
LIMIT 10
```

**good**
```
-- Single Arguments
SELECT client_id
FROM main_summary
WHERE submission_date > '20180101'
LIMIT 10
```
```
-- Multiple Arguments in SELECT and WHERE
SELECT client_id
    , submission_date
FROM main_summary
WHERE submission_date > '20180101'
   AND sample_id = '42'
LIMIT 10
```

## Comas

Las comas para las listas que están divididas por líneas deben ir precedidas de una coma después de un solo espacio. En las declaraciones debe haber
una lista en la misma línea de hasta ~ 120 caracteres.

**bad**

```
SELECT
    client_id,
    sample_id,
    submission_date
FROM main_summary
WHERE
    submission_date > '20180101'
   AND sample_id IN ('a','b','c')
LIMIT 10
```


**good**
```
SELECT client_id
    , sample_id
    , submission_date
FROM main_summary
WHERE submission_date > '20180101'
   AND sample_id IN ('a','b','c')
LIMIT 10
```

## Indentación

Cada bloque de código debe tener indentación junto con cada línea que comience con una palabra clave.


* Los argumentos deben comenzar con el argumento de arriba y de abajo
* Los operadores booleanos deben alinearse con el final de la declaración **WHERE / CASE WHEN** anterior.


Para declaraciones entre paréntesis


* El paréntesis de apertura debe terminar la línea.
* El paréntesis de cierre debe alinearse debajo del primer carácter de la línea que inicia la construcción de varias líneas.
* El contenido de los paréntesis debe tener una sangría de un nivel.


**bad**
```
WITH orders AS (
SELECT
name
, order_date
, SUM(sales) AS daily_sales
FROM orders
WHERE order_date BETWEEN '2021-01-01' AND '2021-01-31'
GROUP BY
name
, order_date
)
```
```
SELECT DISTINCT name
FROM orders
WHERE
name != 'Blue'
AND daily_sales < 100
```

**good**
```
WITH orders AS (
    SELECT name
        , order_date
        , SUM(sales) AS daily_sales
    FROM orders
    WHERE order_date BETWEEN '2021-01-01' AND '2021-01-31'
    GROUP BY name
        , order_date
)


SELECT DISTINCT name
FROM orders
WHERE name != 'Blue'
   AND daily_sales < 100
```

## Rango de fechas


* Asegúrese de que todos los filtros y tablas utilicen un intervalo de fechas coherente en toda la consulta.
* Mantener un formato de datos coherente 
* Evitar lagunas en los datos cuando un evento no se produce en una fecha específica.

## Sea explícito

Al elegir entre sintaxis explícita o implícita, prefiera explícita.

### alias

Incluya siempre la palabra clave AS cuando cree un alias para una variable o nombre de tabla, es más fácil de leer cuando es explícito.

**bad**
```
SELECT date(submission_timestamp) day
FROM telemetry.main
LIMIT 10
```

**good**

```
SELECT date(submission_timestamp) AS day
FROM telemetry.main
LIMIT 10
```

## Joins

En caso de un `JOIN`, utilice `ON` inmediatamente después del `JOIN`. Nunca en el `WHERE`. Y siempre estudie la tabla previamente, para asegurar el uso de
llaves primarias cuando sea posible.

**bad**
```
SELECT orders.name
    , items.description
    , orders.order_date
    , SUM(orders.sales) AS daily_sales
FROM orders AS orders
INNER JOIN items AS items
WHERE order_date BETWEEN '2021-01-01' AND '2021-01-31'
    AND orders.item_nbr = items.item_nbr
GROUP BY orders.name
    , items.description
    , orders.order_date
```
**good**

```
SELECT orders.name
    , items.description
    , orders.order_date
    , SUM(orders.sales) AS daily_sales
FROM orders AS orders
INNER JOIN items AS items
ON orders.item_nbr = items.item_nbr
WHERE order_date BETWEEN '2021-01-01' AND '2021-01-31'
GROUP BY orders.name
    , items.description
    , orders.order_date
```
Cuando utilice una combinación que no sea `INNER JOIN`, incluya el tipo de combinación

**bad**

```
SELECT
    submission_date
    , experiment.key AS experiment_id
FROM telemetry.clients_daily, UNNEST(experiments.key_value) AS experiment -- Implicit Join
WHERE
    submission_date > '2019-07-01'
   AND sample_id = '10'
GROUP BY
    submission_date
    , experiment_id
    , experiment_branch
```

**good**

```
SELECT submission_date
    , experiment.key AS experiment_id
FROM telemetry.clients_daily
CROSS JOIN UNNEST(experiments.key_value) AS experiment
WHERE submission_date > '2019-07-01'
   AND sample_id = '10'
GROUP BY submission_date
    , experiment_id
    , experiment_branch
```

## Lecturas recomendadas

* [Mazur's SQL Style Guide](https://github.com/mattm/sql-style-guide)